In [ ]:
!pip install mlrose

In [ ]:
import numpy as np
import mlrose

In [ ]:
city_number  = 150

In [ ]:
def gen0_func(gen_size,Cities = None):
  if Cities == None:
    Route_Zero = []
    for x in range(city_number):
      Route_Zero.append([np.random.rand()*100 ,np.random.rand()*100])
    #print(Route_Zero)
  else:
    Route_Zero = Cities



  Index_array = []
  for t in range(city_number):
    Index_array.append(t)

  Gen_0 = []
  for y in range(gen_size):
    np.random.shuffle(Index_array)
    local_route = []
    for z in Index_array:
      local_route.append(Route_Zero[z])
    Gen_0.append(local_route)
    
  return Gen_0, Route_Zero

In [ ]:
def eucl_func(a,b):
  return np.sqrt(pow(abs(b[0]-a[0]),2) + pow(abs(b[1]-a[1]),2))

In [ ]:
def route_len(route):
  cycle_len = 0
  for x in range(0,len(route)-1):
    cycle_len = cycle_len + eucl_func(route[x],route[x+1])
  return 1/cycle_len

In [ ]:
def gen_fit(generation):
  gen_len = len(generation)
  trial_fit = []
  for x in generation:
    trial_fit.append(route_len(x))
  return trial_fit

In [ ]:
def selection_prob(fit_for_sel):
  sum_of_fit = 0
  for x in fit_for_sel:
    sum_of_fit = sum_of_fit + x
  
  sel_array = []
  for y in fit_for_sel:
    sel_array.append(y/sum_of_fit)
  
  return sel_array


In [ ]:
def prob_checker(chance):
  prob_rand = np.random.random()
  if prob_rand<=chance:
    return 1
  else:
    return 0

In [ ]:
def sel_index(sel_prob):
  ind_array = []
  loc_sel = sel_prob
  loc_sel_unsort = sel_prob
  loc_sel=sorted(loc_sel_unsort,reverse=True)
  for x in loc_sel:
    for y in range(len(sel_prob)):
      if x == sel_prob[y]:
        ind_array.append(y)
  return ind_array

In [ ]:
def crossover_op(gen_to_cross, sel_ind,crs_prob,seg_len):
  new_gen = gen_to_cross
  if prob_checker(crs_prob):
    R1 = gen_to_cross[sel_ind[0]]
    rand_ind = np.random.randint(0,5)
    if rand_ind == sel_ind[0]:
      while rand_ind == sel_ind[0]:
        rand_ind = np.random.randint(0,5)
    R2 = gen_to_cross[rand_ind]
    segment_pos1 = np.random.randint(0,city_number)
    segment_pos2 = np.random.randint(0,city_number)
    segment_length = seg_len
    segment = R1[segment_pos1:segment_pos1+segment_length]
    Child_R = np.zeros((city_number,2),dtype=np.float32)
    for x in range(len(segment)):
      Child_R[x][0] = segment[x][0]
      Child_R[x][1] = segment[x][1]
    
    counter2 = 0
    for y in R2:
      counter1 = 0
      for z in Child_R:
        if y[0] != z[0] and y[1] != z[1]:
          counter1 = counter1 + 1
      if counter1 == city_number:
        Child_R[segment_length+counter2][0] = y[0]
        Child_R[segment_length+counter2][1] = y[1]
        counter2 = counter2 + 1
        if counter2 == city_number-segment_length:
          break
          
    np.roll(Child_R,segment_pos2)
    for z in range(len(Child_R)):
      new_gen[sel_ind[-1]][z][0] = Child_R[z][0]
      new_gen[sel_ind[-1]][z][1] = Child_R[z][1]

    return new_gen
  else:
    return new_gen

In [ ]:
def mutation_op(gen_to_mutate,mutation_prob):
  mutated_gen = []
  for y in range(len(gen_to_mutate)):
    loc_child = gen_to_mutate[y]
    fin_child = gen_to_mutate[y]
    for x in range(len(loc_child)):
      if prob_checker(mutation_prob) == True:
        rnd_ind = np.random.randint(x,city_number)
        fin_child[x] == loc_child[rnd_ind]
        fin_child[rnd_ind] == loc_child[x]
    mutated_gen.append(fin_child)
  return mutated_gen

In [ ]:
def eucl_len(route):
  cycle_len = 0
  for x in range(0,len(route)-1):
    cycle_len = cycle_len + eucl_func(route[x],route[x+1])
  cycle_len = cycle_len + eucl_func(route[0],route[-1])
  return cycle_len

In [ ]:
def gen_eucl_lengths(generation):
  gen_len = len(generation)
  trial_fit = []
  for x in generation:
    trial_fit.append(eucl_len(x))
  return trial_fit

In [ ]:
def prop1(generation_size,cross_probability,mutation_probability,segment_length,generation = None,cities = None):
  if generation == None:
    GEN_0, Route_Zero = gen0_func(generation_size)
  else:
    GEN_0, Route_Zero = gen0_func(generation_size,cities)
  fitness_coords = mlrose.TravellingSales(coords = Route_Zero)
  problem_fit = mlrose.TSPOpt(length = city_number, fitness_fn = fitness_coords, maximize=False)
  best_state, best_fitness = mlrose.genetic_alg(problem_fit, random_state = 2)

  probs = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
  mutations = [0.01,0.05,0.1,0.15]
  seg_lengths = [10,20,30,40,50]
  min_error = 1e6
  i = 0
  best_params = []
  best_gen = []
  for prob in probs:
    for mutation in mutations:
      for seg_len in seg_lengths:
        i = i + 1
        print('iter:={}'.format(i))
        print('prob={} mutation_prob={} seg_len={}'.format(prob,mutation,seg_len))
        LENGTHS_0 = gen_eucl_lengths(GEN_0)
        FITNESS_0 = gen_fit(GEN_0)
        SELECTION_P_0 = selection_prob(FITNESS_0)
        SELECTION_0 = sel_index(SELECTION_P_0)
        GEN_1 = crossover_op(GEN_0,SELECTION_0,prob,seg_len)
        GEN_1 = mutation_op(GEN_1,mutation)
        LENGTHS_1 = gen_eucl_lengths(GEN_1)
        #print("ORIGINAL LENGTHS:")
        LENGTHS_0.sort()
        #print(LENGTHS_0)
        #print("AFTER GENETIC MODS:")
        LENGTHS_1.sort()
        MIN_LEN = min(LENGTHS_0[0], LENGTHS_1[0])
        params = [prob,mutation,seg_len]
        error = (MIN_LEN - best_fitness)/best_fitness
        print('min_len={}'.format(MIN_LEN))
        print('mlrose:{}'.format(best_fitness))
        print('error={}'.format(error))
        if min_error > error:
          min_error = error
          best_params = params
          best_gen = GEN_1
  print('best_params={}'.format(best_params))
  print('min_error={}'.format(min_error))

  return best_gen#GEN_1

In [ ]:
def prop(generation_size,cross_probability,mutation_probability,segment_length,generation = None,cities = None):
  if generation == None:
    GEN_0 = gen0_func(generation_size)
  else:
    GEN_0 = gen0_func(generation_size,cities)
  LENGTHS_0 = gen_eucl_lengths(GEN_0)
  FITNESS_0 = gen_fit(GEN_0)
  SELECTION_P_0 = selection_prob(FITNESS_0)
  SELECTION_0 = sel_index(SELECTION_P_0)
  GEN_1 = crossover_op(GEN_0,SELECTION_0,cross_probability,segment_length)
  GEN_1 = mutation_op(GEN_1,mutation_probability)
  LENGTHS_1 = gen_eucl_lengths(GEN_1)
  print("ORIGINAL LENGTHS:")
  LENGTHS_0.sort()
  print(LENGTHS_0)
  print("AFTER GENETIC MODS:")
  LENGTHS_1.sort()
  print(LENGTHS_1)
  return GEN_1

In [ ]:
def run(iterations,INPUT = None):
  loc_generation = None
  for x in range(iterations):
    if x == 0:
      loc_generation = prop1(10,0.6,0.05,11,loc_generation,INPUT)
    else:
      loc_generation = prop1(10,0.6,0.05,11,loc_generation)
  return loc_generation

In [ ]:
#INPUT = []
#for x in a:
#  INPUT.append([x[0]/100,x[1]/100])

In [ ]:
#INPUT